In [ ]:
# Extract SO-CI information from MOLPRO outputs for atoms
#   Read exptl data from Excel file, combine with weights to get E_so
#   The experimental Excel file is generated by get_NIST_atomic_data.ipynb
# Add a fourth way to get E_so:  by fitting term energies to exptl level energies
#   (off-diagonal elements of the spin-orbit matrix are from a Molpro SO-CI calculation)
# KKI 2/5/2024
import re, sys, os
import numpy as np
import pandas as pd
import scipy
#import matplotlib.pyplot as plt
#from collections import Counter
#from sklearn.cluster import KMeans

import chem_subs as chem
import molpro_subs as mpr

pd.set_option('display.max_rows', None)
np.set_printoptions(suppress=True)

### Specify Molpro SO-CI output file

In [ ]:
fdir = r'C:\Users\irikura\OneDrive - NIST\Karl\atomic_SOC\calculations\B_I'
fname = 'b_2P_55_actz.pro'

In [ ]:
fsoc = os.sep.join([fdir, fname])
print(f'Reading MOLPRO file "{fsoc}"')
atom = mpr.stoichiometry(fsoc)
charge = int(np.rint(mpr.total_charge(fsoc, verbose=True)))
print(f'The atom is {atom} with charge {charge}')
# append charge, to match sheet names in exptl data file
if charge > 0: 
    atom += '+'
elif charge < 0:
    atom += '-'
if abs(charge) > 1:
    atom += f'{abs(charge)}'

In [ ]:
SOCI = mpr.fullmatSOCI(fsoc, atom=True)
labels_ordinated = False  # flag to prevent multiple (1)(1)(1) etc. 

In [ ]:
caslist, lineno_cas = mpr.readMULTI(fsoc, PG='Ci', linenum=True, atom=True)
CAS = caslist[-1]   # assume the last CASSCF to be the relevant one
#CAS.results = relabel_CAS_by_energy(CAS.results)
CAS.results

In [ ]:
SOCI.dfci

In [ ]:
print(f'CASSCF active space is ({SOCI.cas.nactel()}/{SOCI.cas.nactorb()})')
# Get the basis set
rx_bs = re.compile(' SETTING BASIS\s+=\s+(\S+)')
with open(fsoc, 'r') as F:
    for line in F:
        m = rx_bs.match(line)
        if m:
            basisset = m.group(1)
print(f'Basis set is {basisset}')
dynw = SOCI.cas.dynw()
caswts = SOCI.cas.weights(rescale=True)
if dynw:
    print(f'Dynamical weighting with dynw={dynw}')
    print('Final weights:')
else:
    print('CASSCF state weights (subject to rounding error):')
for wts in caswts:
    print('    ', np.round(wts, 2))

In [ ]:
SOCraw = SOCI.vals.min()
print(f'From lowest level and lowest uncoupled term energy, raw theoretical SOCraw = {SOCraw:.1f} cm-1')

In [ ]:
# Read experimental energy levels
if charge >= 0:
    atstr = atom + '_' + 'I' * (charge + 1)
else:
    # anion
    atstr = atom + '_neg'
fxl = f'{atstr}_exptl_levels.xlsx'
xlpath = os.sep.join([fdir, fxl])
dfexpt = pd.read_excel(xlpath)
print(f'Experimental energy levels read from {fxl}')

In [ ]:
# Find the number of decimal places in the level energies
Ecol = 'Level (cm-1)'  # the exptl energy column
ndecim = 0
for e in dfexpt[Ecol]:
    words = str(e).split('.')
    # count numeric digits
    n = sum(c.isdigit() for c in words[-1])
    ndecim = max(n, ndecim)
print(f'Experimental energies are provided to {ndecim} decimal digits')

In [ ]:
# display formatting
fmt = {'Eshift': '{:.1f}', 'Pct': '{:.3f}', 'degen': '{:.0f}'}
fmt[Ecol] = '{:.' + str(ndecim) + 'f}'
for col in ['J', 'Ecalc', 'E_dif', 'Erel', 'Eshift', 'err', 'Eterm', 'cm-1', 'fitted',
           'wmean', 'wstds', 'uwmean', 'uwstds', 'change']:
    fmt[col] =  fmt['Eshift']
for col in ['dif', 'Theory', 'ecm', 'SOC', 'RMSE']:
    fmt[col] = '{:.2f}'

In [ ]:
dfterm = SOCI.average_terms(be_close=['Energy', 'Edav', 'Eref', 'dipZ', 'C0'], 
                            atom=True, always=True)
# drop the dipZ column
dfterm.drop(columns=['dipZ'], inplace=True)
print(f'Averaged terms from MRCI ({fname}):')
dfso = SOCI.assign_atomic_J(quiet=True)  # create SOCI.dfso
Egl = SOCI.dfso.E.min()  # energy of ground level
dfterm['Erel'] = (dfterm.Edav - Egl) * chem.AU2CM
display(dfterm.style.format(fmt))
print('"ecm"  is relative to the lowest term (cm-1)')
print('"Erel" is relative to the ground level')

In [ ]:
irreps_ci = set(SOCI.dfci.Irrep)
if (SOCI.PG == 'Ci') and (len(irreps_ci) == 1):
    if 1 in irreps_ci:
        parity = 'even'
    else:
        parity = 'odd'
else:
    # ask user for parity of interest
    parity = input('Please choose "even" or "odd" parity: ')
print(f'Experimental states will be restricted to parity = {parity}')

In [ ]:
# Delete any ionization limit
ilim = dfexpt[dfexpt.Term == 'Limit'].index.min()
# delete the "Limit" row and everything past it
n1 = len(dfexpt)
dfexpt = dfexpt.truncate(after=ilim-1)
n2 = len(dfexpt)
if n2 < n1:
    print(f'Discarding {n1-n2} ionized or metastable states')
oddstr = r'\*$|°' # regex to identify terms of odd parity
dfeven = dfexpt[~(dfexpt.Term.str.contains(oddstr) | dfexpt.Configuration.str.contains(oddstr))].copy()
dfodd = dfexpt[dfexpt.Term.str.contains(oddstr) | dfexpt.Configuration.str.contains(oddstr)].copy()
print(f'{len(dfexpt)} experimental levels ({len(dfeven)} even and {len(dfodd)} odd)')
# Select by parity
if parity == 'even':
    # discard odd levels ('Term' field ends with '*')
    dfexpt = dfeven.copy()
elif parity == 'odd':
    dfexpt = dfodd.copy()
else:
    chem.print_err('', f'Parity of "{parity}" is not recognized')
n3 = len(dfexpt)
print(f'{n3} levels accepted for parity = {parity}')
# Reject bad values of J
for i in dfexpt.index:
    try:
        chem.halves_to_float(dfexpt.loc[i, 'J'])
    except ValueError:
        dfexpt.at[i, 'J'] = np.nan
nbad = dfexpt.J.isna().sum()
if nbad:
    print(f'** Rejecting {nbad} levels with malformed J values')
    dfexpt = dfexpt.dropna()
    n4 = len(dfexpt)
    print(f'{n4} level retained')
# Assign unique term symbols
if not labels_ordinated:
    dfexpt = chem.unique_labels_exptl_terms(dfexpt, verbose=True, always=True)
    labels_ordinate = True
dfexpt

In [ ]:
def match_expt_theory_simple(dfexpt, dftheory):
    # Match exptl and theoretical levels, based upon leading term
    #    (will fail if term assignments differ)
    # Return a DataFrame containing both theory and expt
    if 'degen' not in dfexpt.columns:
        # Add degeneracy
        dfexpt['degen'] = (2 * dfexpt.J.apply(chem.halves_to_float)) + 1
    dfcomp = dfexpt.copy()
    dfcomp['Tcalc'] = ''  # term assignment in computation
    dfcomp['termwt'] = ''
    dfcomp['Ecalc'] = np.nan
    for i, row in dftheory.iterrows():
        term = row.Lead
        J = chem.halves_to_float(row.J)
        #print(f'theory row {i} with J = {J} for term {term}')
        for ix, rowx in dfexpt.iterrows():
            xTerm = rowx.uTerm
            xJ = chem.halves_to_float(rowx.J)
            #print(f'    expt row {ix} with J = {xJ} for term {xTerm}')
            if (xJ == J) and (term == xTerm):
                if not np.isnan(dfcomp.at[ix, 'Ecalc']):
                    print('Already paired!', display(rowx.to_frame().T))                   
                else:
                    dfcomp.at[ix, 'Ecalc'] = row.Erel
                    dfcomp.at[ix, 'Tcalc'] = term
                    dfcomp.at[ix, 'termwt'] = row.termwt
    dfcomp['err'] = dfcomp.Ecalc - dfcomp[Ecol]
    return dfcomp

In [ ]:
dfdiff = match_expt_theory_simple(dfexpt, dfso)
# Convert str values of J to float
dfdiff['J'] = dfdiff.J.apply(chem.halves_to_float)
warnThresh = 1000  # highlight errors larger than this (cm-1)
# drop rows with NaN (no matching level in the calculation)
dfdiff.dropna(axis=0, inplace=True)
print(f'Please inspect the following pairing of theory ("Ecalc") with expt ("{Ecol}")')
print('Disagreements in term assignments are highlighted in red')
print(f'Errors > {warnThresh} cm-1 are highlighted in yellow')
selcols = ['Configuration', 'uTerm', 'J', Ecol, 'Tcalc', 'Ecalc', 'err']
display(dfdiff[selcols].style.apply(lambda x: ["background: yellow" if abs(v) > warnThresh else "" for v in x], 
              subset=pd.IndexSlice[['err']]).apply(lambda x: (x != dfdiff['uTerm']).map({True: "background-color: red; \
              color: white", False: ""}), subset=['Tcalc']).format(fmt))

In [ ]:
# No theoretical calculations are needed to use eq. (1)
xterms = []  # list of term labels
eterms = []  # list of term energies
for term in dfdiff.uTerm:
    if term not in xterms:
        xterms.append(term)
for Term in xterms:
    subdf = dfexpt[dfexpt.uTerm == Term]
    emean = np.dot(subdf.degen, subdf[Ecol]) / subdf.degen.sum()
    eterms.append(emean)
dfeq1 = pd.DataFrame({'Term': xterms, 'Eterm': eterms}).sort_values('Eterm').reset_index(drop=True)
print('Term energies (cm-1) using eq. (1) (experimental data with naive model)')
display(dfeq1.style.format(fmt))
SOC1 = -1 * np.round(dfeq1.at[0, 'Eterm'], 3)
lowterm = dfeq1.at[0, 'Term']
print(f'The term of lowest energy is \t{lowterm} \twith SOC1 = {SOC1} cm-1')
levterm = dfexpt.uTerm.values[0]

target = levterm

if levterm != lowterm:
    # The lowest term is not the leading term in the lowest level
    SOC1alt = SOC1
    SOC1 = -1 * np.round(dfeq1[dfeq1.Term == levterm]['Eterm'].values[0], 3)
    print(f'The lowest level belongs to \t{levterm} \twith SOC1 = {SOC1} cm-1')
print()
print(f'Term {target} is selected for further SOC calculation')
print('    to change this, assign the variable "target" to another term in the table above')

In [ ]:
def term_energy_from_levels(df, term):
    # Given a DataFrame with the right columns ['J', 'termwt', Ecol],
    #   where 'Ecol' is the header for the column of level energies,
    # Return the term's average energy as derived from the levels
    global SOCI
    # find index for term 'term'
    iterm = SOCI.dfterm[SOCI.dfterm.Term == term].index[0]
    termwt = []
    for twt in df.termwt:
        try:
            termwt.append(twt[iterm])
        except IndexError:
            # assume scalar
            termwt.append(twt)
    termwt = np.array(termwt)
    #termwt = np.array([twt[iterm] for twt in df.termwt])
    degen = 2 * df.J.values + 1
    dweight = degen * termwt  # total weight, including degeneracies
    Eterm = np.dot(df[Ecol], dweight) / dweight.sum()
    return Eterm

In [ ]:
target

In [ ]:
# Use experimental level energies via eq. (2)
Eterm = term_energy_from_levels(dfdiff, target)
SOC2 = -Eterm
print('Applying eq. (2) (experimental energies and theoretical term weights)')
print(f'For term {target}, SOC2 = {SOC2:.2f} cm-1')

In [ ]:
print(f'Molpro source file: {fname}\n')
print(f'Alternative values for E_so[{target}] of atom {atom}:')
print('-' * 25)
print('{:12s} {:.2f} cm-1'.format('eq (1)', SOC1))
print('{:12s} {:.2f} cm-1'.format('raw theory', SOCraw))
print('{:12s} {:.2f} cm-1'.format('eq (2)', SOC2))
print('-' * 25)

In [ ]:
def term_distrib(term, df):
    # return the weights (including 2J+1) of term in levels
    global SOCI
    itarget = SOCI.dfterm[SOCI.dfterm.Term == term].index[0]
    wt = []
    for twt in df.termwt:
        try:
            wt.append(twt[itarget])
        except IndexError:
            # assume scalar
            wt.append(twt)
    wt = np.array(wt)
    #wt = [twt[itarget] for twt in df.termwt]  # without 2J+1 weighting
    wt = wt * (2*df.J + 1)
    return wt

In [ ]:
display(dfdiff)

In [ ]:
# Term energy errors as inferred from all levels
dftermerr = pd.DataFrame(columns=['Term', 'wmean', 'wstds'])
termlist = []
wmean = []
wstds = []
# also consider unsigned (absolute value) errors
uwmean = []
uwstds = []
for term in set(dfdiff.uTerm):
    termlist.append(term)
    weights = term_distrib(term, dfdiff).values
    m, s = chem.weighted_mean(dfdiff.err, weights)
    wmean.append(m)
    wstds.append(s)
    uerr = np.abs(dfdiff.err.values)
    um, us = chem.weighted_mean(uerr, weights)
    uwmean.append(um)
    uwstds.append(us)
dftermerr['Term'] = termlist
dftermerr['wmean'] = wmean
dftermerr['wstds'] = wstds
dftermerr['uwmean'] = uwmean
dftermerr['uwstds'] = uwstds

if dftermerr.isnull().values.any():
    print('*** Some terms are missing ***')
    print('Try decreasing the energy maximum ("termcut")')
else:
    print('Errors in term energies (cm-1) as inferred from full level distribution')
    print('    (not only levels where leading)')
    # default order same as experimental terms
    dftermerr.Term = pd.Categorical(dftermerr.Term, xterms)
    dftermerr = dftermerr.sort_values('Term')
dftermerr.sort_values('uwmean').style.format(fmt)

In [ ]:
print(f'Distribution of term "{target}" among levels:')
thrsh = 1.e-6
dfdistrib = dfso.copy()
dfdistrib[target] = term_distrib(target, dfso)
# remove rows with negligible weights
dfdistrib = dfdistrib[dfdistrib[target] > thrsh]
dfdistrib.drop(['termwt', 'Composition'], axis=1, inplace=True)
display(dfdistrib.sort_values(target, ascending=False).style.format(fmt))
print(f'Total weight of {target} = {dfdistrib[target].sum():.3f}')

In [ ]:
# Don't attempt level-fitting by default
1/0

## Fitting of term energies to experimental level energies

In [ ]:
def levels_from_term_energies(term_order, term_energies):
    # Install the term energies along the SOCI.matrix diagonal and rediagonalize
    #   'term_order' is array of term symbols
    #   'term_energies' is array of corresponding energies (cm-1)
    # Return nothing
    #global SOCI
    #   'SOCI' is a fullmatSOCI() object and is modified
    term_dict = dict(zip(term_order, term_energies))
    newdiag = SOCI.matrix.diagonal().copy()
    for ibs in range(len(newdiag)):
        j = SOCI.sob_ici[ibs]
        term = SOCI.mrci[j].Term
        # install the new energy for the term
        newdiag[ibs] = term_dict[term]
    # update the matrix
    SOCI.fill_diagonal(newdiag)
    SOCI.diagonalize(store=True, vectors=True, sortval=False)  # update the SOCI() object
    SOCI.assign_atomic_J(quiet=True)
    return

In [ ]:
def rmserr(dfcomp):
    # Given a matched-up comparison of expt and theory, return the
    #   rms error in level energies
    # Weighted with degeneracies
    rmse = np.dot(dfcomp.err**2, dfcomp.degen.astype(float))
    rmse /= dfcomp.degen.sum()
    rmse = np.sqrt(rmse)
    return rmse
def obj_fun(exc_terme):
    # Given only excited term energies (assuming ground=0)
    #   return the RMSE
    global term_order, dfexpt, SOCI
    terme = [0] + list(exc_terme)  # the fixed energy of the ground term is 0
    levels_from_term_energies(term_order, terme)
    dfcomp = match_expt_theory_simple(dfexpt, SOCI.dfso)
    # drop rows with NaN (no matching level in the calculation)
    dfcomp.dropna(axis=0, inplace=True)
    rmse = rmserr(dfcomp)
    return rmse

In [ ]:
# Create global 'term_order' 
term_order = dfterm.Term.values
term_energies = dfterm.ecm.values

In [ ]:
print('Initial comparison with expt, before fitting')
rmse0 = rmserr(dfdiff)
print(f'RMSE = {rmse0:.1f} cm-1')
display(dfdiff.style.format(fmt))

In [ ]:
def freport(xvec):
    # callback function to monitor minimization
    # this version provides more info, at slightly increased cost
    freport.counter += 1
    print(f'{freport.counter:5d}', end='')
    for x in xvec:
        print(f'   {x:.1f}', end='')
    fun = obj_fun(xvec)
    print(f'  RMSE = {fun:.1f}')
    return
freport.counter = 0

In [ ]:
# Minimize the RMSE (may take hours)
exc_terme = list(term_energies)[1:]  # only excited terms; assume ground term = 0 energy
result = scipy.optimize.minimize(obj_fun, exc_terme, method='Nelder-Mead', callback=freport)

In [ ]:
if result.success:
    print(f'Minimization complete in {result.nit} iterations with {result.nfev} evaluations')
    print(f'RMSE = {result.fun:.1f} after Nelder-Mead')
    rmse = result.fun
    print('Fitted levels')
    fit_terme = [0] + list(result.x)
    levels_from_term_energies(term_order, fit_terme)
    dfcomp = match_expt_theory_simple(dfexpt, SOCI.dfso)
    # drop rows with NaN (no matching level in the calculation)
    dfcomp.dropna(axis=0, inplace=True)
    rmse = rmserr(dfcomp)
    # Convert str values of J to float
    dfcomp['J'] = dfcomp.J.apply(chem.halves_to_float)
    display(dfcomp.style.format(fmt))
    print(f'RMSE = {rmse:.1f} cm-1 compared with initial RMSE = {rmse0:.1f}')
else:
    print('*** Failure ***')
    print(result)

In [ ]:
# Add to Term DataFrame
print('Fitted term energies; "Erel" is relative to the lowest level')
dfterm['fitted'] = np.nan
for term, terme in zip(term_order, fit_terme):
    dfterm.loc[dfterm.Term == term, 'fitted'] = terme
Eshift = SOCI.dfso.Eshift.min()  # energy of ground level relative to lowest term
dfterm['Erel'] = (dfterm.fitted - Eshift) # relative to ground level
dfterm['change'] = dfterm.fitted - dfterm.ecm
dfterm.style.format(fmt)

In [ ]:
SOCfit = -dfterm.loc[dfterm.Term == target, 'Erel'].values[0]

In [ ]:
print(f'Molpro source file: {fname}\n')
print(f'Alternative values for E_so[{target}] of atom {atom}:')
print('-' * 25)
print('{:12s} {:.1f} cm-1'.format('eq (1)', SOC1))
print('{:12s} {:.1f} cm-1'.format('raw theory', SOCraw))
print('{:12s} {:.1f} cm-1'.format('eq (2)', SOC2))
print('{:12s} {:.1f} cm-1'.format('term fitting', SOCfit))
print('-' * 25)